## Intro

The purpose of today's class is to explore data using **interactive visualizations**. Interactivity is a key part of modern dataviz. It's a way to allow users of your visualizations get their own feel for the data ... to create richer visualization, where people who use your work can expose more of the data by exploring.

I've also got a little video of "practical updates"

[![IMAGE ALT TEXT HERE](https://github.com/suneman/socialdata2024/blob/main/files/thumbnail_week6.png?raw=true)](https://panopto.dtu.dk/Panopto/Pages/Viewer.aspx?id=13f93c21-8675-4c5a-bbad-b12a00adaa1c)

## Part 1: Video Lectures and Reading

Starting this week, we'll be playing around with *explanatory data visualization*. Roughly speaking this means using data visualization to communicate your results to others. Thus, there are new things to think about. 

Until today we have worked with static data visualization. However, exploratory data analysis means to be able to explore the multi-faceted nature of data and *interactive dataviz* is a handy tool to do it! It allows to play with the data: Toggle the view. Zoom. Drag. Show more details. All those things. Those are a key part of modern data visualization. 

The video below provides context about these points.

We start with the video and then read a bit from a scientific article about types of explanatory dataviz. (*The video is from an old version of the class that used D3, so just ignore those parts.*).

[![IMAGE ALT TEXT HERE](https://img.youtube.com/vi/yHKYMGwefso/0.jpg)](https://www.youtube.com/watch?v=yHKYMGwefso)

> **Note**Keep your answers to the questions short and sweet!
> *Exercises*: Explanatory data visualization. <font color=gray>It's OK to use LLMs here if you can</font>.
> * What are the three key elements to keep in mind when you design an explanatory visualization?
> * In the video I talk about (1) *overview first*,  (2) *zoom and filter*,  (3) *details on demand*. 
>   - Go online and find a visualization that follows these principles (don't use one from the video). 
>   - Explain how your video achieves (1)-(3). It might be useful to use screenshots to illustrate your explanation.
> * Explain in your own words: How is explanatory data analysis different from exploratory data analysis?

Explanatory data analysis is about you telling the story of some data. It is showing relations and what some data might really indicate.

Exploratory data analysis is more like a GUI. It is making a framework for a user to explore the data on your own. However you shape the tools, otherwise they would just go into something like Python, themselves, so you have to be aware what questions the user might have.

## Part 2: Interactive visualizations with Bokeh



To really master interactive visualizations, you will need to work with JavaScript, especially [D3](https://d3js.org). Given the limited time available for this class, we can't squeeze that in. But luckily Python has some pretty good options for interactive visualizations. You can find a range of different options [here](https://mode.com/blog/python-interactive-plot-libraries/).

Today, we'll explore [`Bokeh`](https://docs.bokeh.org/en/latest/), which provides lots of nice interactive funtionalities to Python. To work with Bokeh, we first set up our system:

1. If you haven't installed it yet please do so. You can simply follow [these steps](https://docs.bokeh.org/en/latest/docs/first_steps/installation.html)

2. To include Bokeh in your notebooks you can follow the [Bokeh: Using with Jupyter](https://docs.bokeh.org/en/latest/docs/user_guide/output/jupyter.html#jupyter) guide. Come back to this one when you need it

3. We aim to give you a gentle start with Bokeh and I am going to include more example code than usual in the follwing.
   * **HINT 1**: If you're not an experienced Python user, I recommend going to the [official user's guide](https://docs.bokeh.org/en/latest/docs/user_guide.html#userguide) and working through it. Start by clicking "Introduction" in the linked page. That page has a glossary, a section on output methods, stuff on settings, and interfaces that you can scroll through. The next page *Basic Plotting* where the action is. Spend some time working through that.
   * **HINT 2**: And by "working through it", I mean copy, paste, and run the code in your own notebook. 

Ok. Let's get started. First a general announcement on the data.

> **Announcement**
> * During this entire lecture, as always, we are going to work with the SF Crime Data. 
> * We will use data for the **period 2010-2017***.


Now, to get you in the mood here's a little gif to illustrate what the goal of this exercise is:

![Movie](https://github.com/suneman/socialdata2024/blob/main/files/week6_1.gif?raw=true)

If the gif isn't displaying on your system, you can download it [here](https://github.com/suneman/socialdata2024/blob/main/files/week6_1.gif) and display locally.


> ***Exercise***: Recreate the results from **Week 2** as an interactive visualisation (shown in the gif). To complete the exercise, follow the steps below to create your own version of the dataviz.

### Data prep

A key step is to set up the data right. So for this one, we'll be pretty strict about the steps. The workflow is

1. Take the data for the period of 2010-2017 and group it by hour-of-the-day.
2. We would like to be able to easily compare how the distribution of crimes differ from each other, not absolute numbers, so we will work on *normalized data*:
    * To normalise data for within a crime category you simply to devide the count for each hour by the total number of this crime type. (To give a concrete example in the `ASSAULT` category, take the number of assault-counts in 1st hour you should devide by the total number of assaults, then you devide number of assaults in 2nd hour by the total number of assaults and so on)
    *  Your life will be easiest if you organize your dataframe as shown in [this helpful screenshot](https://github.com/suneman/socialdata2024/blob/main/files/W6_Part2_data.png).



In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

SAVE_PATH = "plots/Week6/"

focuscrimes = set(['WEAPON LAWS', 'PROSTITUTION', 'DRIVING UNDER THE INFLUENCE', 'ROBBERY', 'BURGLARY', 'ASSAULT', 'DRUNKENNESS', 'DRUG/NARCOTIC', 'TRESPASS', 'LARCENY/THEFT', 'VANDALISM', 'VEHICLE THEFT', 'STOLEN PROPERTY', 'DISORDERLY CONDUCT'])
NUM_FOCUS_CRIMES = len(focuscrimes)

In [2]:
# *** Load and filter the data ***
FILENAME = "data/Police_Department_Incident_Reports__Historical_2003_to_May_2018_20240130.csv"
crime_df = pd.read_csv(FILENAME)
crime_df = crime_df[crime_df["Date"].str.contains("2018") == False] # Remove 2018 data


print(crime_df.head())

             PdId  IncidntNum  Incident Code       Category  \
0   4133422003074    41334220           3074        ROBBERY   
1   5118535807021    51185358           7021  VEHICLE THEFT   
2   4018830907021    40188309           7021  VEHICLE THEFT   
3  11014543126030   110145431          26030          ARSON   
4  10108108004134   101081080           4134        ASSAULT   

                Descript DayOfWeek        Date   Time PdDistrict Resolution  \
0  ROBBERY, BODILY FORCE    Monday  11/22/2004  17:50  INGLESIDE       NONE   
1      STOLEN AUTOMOBILE   Tuesday  10/18/2005  20:00       PARK       NONE   
2      STOLEN AUTOMOBILE    Sunday  02/15/2004  02:00   SOUTHERN       NONE   
3                  ARSON    Friday  02/18/2011  05:27  INGLESIDE       NONE   
4                BATTERY    Sunday  11/21/2010  17:00   SOUTHERN       NONE   

   ... Fix It Zones as of 2017-11-06  2 2  DELETE - HSOC Zones 2 2  \
0  ...                                NaN                      NaN   
1  ...

In [3]:
# *** Group by time of day and category ***

FOCUS_DATASET = True

# Convert date and time to datetime
crime_df["DateTime"] = pd.to_datetime(crime_df["Date"].astype(str) + ' ' + crime_df["Time"].astype(str))

if FOCUS_DATASET:
    crime_df = crime_df[crime_df["Category"].isin(focuscrimes)]

# Group by time of day and category 
time_day_df = crime_df.groupby([crime_df["DateTime"].dt.hour, "Category"])
time_day_df = time_day_df.size().unstack()

# and rename rows to hour
ROW_NAME = "Hour"
time_day_df.index.name = ROW_NAME
time_day_df.columns.name = "Category"


print(time_day_df.head())

Category  ASSAULT  BURGLARY  DISORDERLY CONDUCT  DRIVING UNDER THE INFLUENCE  \
Hour                                                                           
0            8878      3386                 517                          705   
1            8074      2186                 380                          647   
2            7258      2358                 341                          642   
3            3557      2412                 216                          286   
4            2216      2143                 158                          106   

Category  DRUG/NARCOTIC  DRUNKENNESS  LARCENY/THEFT  PROSTITUTION  ROBBERY  \
Hour                                                                         
0                  3686          817          18568          1342     2908   
1                  2242          774          11481          1041     2987   
2                  1817          656           7505           771     3129   
3                  1235          258           46

In [4]:
# Normalize the data 
time_day_normal_df = time_day_df.div(time_day_df.sum(axis=0), axis=1)

print(time_day_normal_df.head())


Category   ASSAULT  BURGLARY  DISORDERLY CONDUCT  DRIVING UNDER THE INFLUENCE  \
Hour                                                                            
0         0.054414  0.038057            0.052551                     0.126799   
1         0.049486  0.024570            0.038626                     0.116367   
2         0.044484  0.026503            0.034662                     0.115468   
3         0.021801  0.027110            0.021956                     0.051439   
4         0.013582  0.024087            0.016060                     0.019065   

Category  DRUG/NARCOTIC  DRUNKENNESS  LARCENY/THEFT  PROSTITUTION   ROBBERY  \
Hour                                                                          
0              0.031680     0.084436       0.039986      0.081566  0.054440   
1              0.019269     0.079992       0.024724      0.063271  0.055919   
2              0.015616     0.067797       0.016162      0.046861  0.058577   
3              0.010614     0.026664 

If you've followed these steps, your data should be ready! Take a moment to celebrate. Below, we follow the [Bokeh guide for categorical data](https://docs.bokeh.org/en/latest/docs/user_guide/basic/bars.html). You can either <font color=gray>use some kind of LLM support to make it happen, it's OK to go nuts and ask away</font> or you can follow the old-school hints from last year:


1. First, let's convert our **Pandas Dataframe** to **Bokeh ColumnDataSource**:
   
  > ```python
  > source = ColumnDataSource(your_processed_dataframe)
  > ## it is a standard way to convert your df to bokeh
  > ```

2. We also need to create an empty figure (we will add our stuff here later on). Mini sub-exercise: Find the a guide how to define a figure in Bokeh online. Here is a little help:
  
  > ```python
  > p = figure(...., x_range = FactorRange(factors=hours), ...) 
  > #p is a standard way to call figures in Bokeh
  > #make sure to add x_range. In my case hours is a list on the form ['1', '2', '3' ... , '24']
  > #read up on the FactorRange in the guide
  > #do not forget to add other attributes to the figure, e.g. title, axis names and so on
  > ```

3. Now we are going to add the bars. In order to do so, we will use **vbar** (see the guide for help):
  
  > ```python
  > bar ={} # to store vbars
  > ### here we will do a for loop:
  > for indx,i in enumerate(focuscrimes):
  >     bar[i] = p.vbar(x='name_of_the_column_that_contain_hours',  top=i, source= src, 
  >                     ### we will create a vbar for each focuscrime
  >                     legend_label=i,  muted_alpha=..., muted = ....) 
  > #i stands for a column that we use, top=y; we are specifying that our numbers comes from column i
  > #read up what legend_label, muted and muted_alpha do... you can add more attributes (you HAVE TO)
  > ```

4. The last thing to do is to make your legend interactive and display the figure:
  
  > ```python
  > p.legend.click_policy="mute" ### assigns the click policy (you can try to use ''hide'
  > show(p) #displays your plot
  > ```

5. You will notice that the legend appears in the middle of the figure (and it occludes some of the data). In order to fix this look into [this guide](https://stackoverflow.com/questions/26254619/position-of-the-legend-in-a-bokeh-plot) as a start. Below are some code snippets that you can use to deal with this problem (but read the guide first):
  
  > ```python
  > items = [] ### for the custom legend // you need to figure out where to add it
  > items.append((i, [bar[i]])) ### figure where to add it
  > legend = Legend(items=..., location=.....) ## figure where to add it
  > p.add_layout(...., ...) ## figure where to add it
  > ### if you read the guide, it will make sense
  > ```

Now you should be able to recreate this amazing visualisation.


**EXTRA feature**: If you're interested in detailed instructions for more Bokeh visualizations for your final project, you can find more inspiration **[here](https://github.com/suneman/socialdata2021/blob/main/lectures/Week8_extra_bokeh.ipynb)**.

In [5]:
import bokeh.plotting as bp
from bokeh.models import ColumnDataSource, HoverTool, FactorRange, Legend
from bokeh.palettes import Category20c
from bokeh.transform import factor_cmap
from bokeh.io import output_file, show, output_notebook
from bokeh.layouts import gridplot

output_notebook()

Loading BokehJS ...

In [7]:
source = ColumnDataSource(time_day_normal_df)

output_file(SAVE_PATH + "crimes_by_time_of_day.html")

# Make the plot
hours = [f"{i:02}:00" for i in range(24)]
p = bp.figure(
    x_range=FactorRange(factors = hours),
    height=400,
    width=1200,
    title="Crimes by time of day",
    toolbar_location="above",
    )

# Make the legend (interactive *thingy*)
bar = {}
items = [None] * NUM_FOCUS_CRIMES # <- to store the legend items

for indx, category in enumerate(focuscrimes):
    bar[category] = p.vbar(
        x=ROW_NAME,
        top=category,
        source=source,
        width=0.9,
        color=Category20c[NUM_FOCUS_CRIMES][indx],
    )
    items[indx] = (category, [bar[category]])

legend = Legend(items=items, location=(0, 0), click_policy="mute")

p.add_layout(legend, "left")
show(p)

## Part 3: Narrative Dataviz

Let's finish up with some reading

*Reading*: [Narrative Visualization: Telling Stories with Data](http://vis.stanford.edu/files/2010-Narrative-InfoVis.pdf) by Edward Segel and Jeffrey Heer. We'll read section 1-3 today. (And the rest a bit later).

When you get to section 3 it's fun to open up the examples mentioned by the authors in a browser and explore them as you read the text. 

> *Exercise*: Answer a couple of questions about the paper.
> 
> * What is the *Oxford English Dictionary's* defintion of a narrative?
> * What is your favorite visualization among the examples in section 3? Explain why in a few words.